<a href="https://colab.research.google.com/github/ProfAI/machine-learning-modelli-e-algoritmi/blob/main/1%20-%20L'algoritmo%20Gradient%20Descent/gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L'algoritmo Gradient Descent

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams["figure.figsize"] = (16,10)
#sns.set_theme()

In [ ]:
RANDOM_SEED = 0

## Regressione

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X, y = make_regression(n_samples=100, n_features=50, bias=5.0, noise=20.0,random_state=RANDOM_SEED)
#plt.scatter(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=RANDOM_SEED)

In [ ]:
model = SGDRegressor(max_iter=1000, 
                     penalty="elasticnet", 
                     alpha=0.01, 
                     l1_ratio=0.9, 
                     learning_rate="adaptive",
                     #eta0=0.001 
                     )

model.fit(X_train, y_train)

SGDRegressor(alpha=0.01, l1_ratio=0.9, learning_rate='adaptive',
             penalty='elasticnet')

In [ ]:
def evaluate(model, data):
    x, y = data
    y_pred = model.predict(x)
    print(f"MSE = {mean_squared_error(y, y_pred)}")
    print(f"R2 = {r2_score(y, y_pred)}")

In [ ]:
evaluate(model, (X_train, y_train))

MSE = 49.16965214187807
R2 = 0.9976109433229705


In [ ]:
evaluate(model, (X_test, y_test))

MSE = 657.2193241144546
R2 = 0.9563281804500007


## Classificazione

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
X, y = make_classification(n_samples=100, n_features=30, n_informative=30, n_redundant=0, n_repeated=0, n_classes=2, random_state=RANDOM_SEED)

In [ ]:
def evaluate(model, data):
    x, y = data
    y_pred = model.predict(x)
    print(f"ACCURACY = {accuracy_score(y, y_pred)}")
    print(f"LOG LOSS = {log_loss(y, y_pred)}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=RANDOM_SEED)

In [ ]:
model = SGDClassifier(max_iter=1000, 
                     penalty="elasticnet", 
                     alpha=0.1, 
                     l1_ratio=0.9, 
                     learning_rate="adaptive",
                     eta0=0.01,
                     random_state=RANDOM_SEED 
                     )

model.fit(X_train, y_train)

SGDClassifier(alpha=0.1, eta0=0.01, l1_ratio=0.9, learning_rate='adaptive',
              penalty='elasticnet', random_state=0)

In [ ]:
model = SGDClassifier(random_state=RANDOM_SEED)
model.fit(X_train, y_train)

SGDClassifier(random_state=0)

In [ ]:
evaluate(model, (X_train, y_train))

ACCURACY = 0.8857142857142857
LOG LOSS = 3.947334422128661


In [ ]:
evaluate(model, (X_test, y_test))

ACCURACY = 0.7666666666666667
LOG LOSS = 8.059181091717525
